# Colored ICP registration

In [1]:
import open3d as o3d
import numpy as np
import copy

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
demo_pcds = o3d.data.DemoICPPointClouds()
source = o3d.io.read_point_cloud("../output/cuadra/pcd/IMG_1701.ply")
target = o3d.io.read_point_cloud("../output/cuadra/pcd/IMG_1703.ply")

In [3]:
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    # source_temp.paint_uniform_color([1, 0.706, 0])
    # target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    # o3d.visualization.draw_plotly([source_temp, target_temp])
    o3d.visualization.draw_geometries([source_temp, target_temp])

In [4]:
trans_init = np.eye(4) # identity matrix / no transformation

In [5]:
draw_registration_result(source, target, trans_init)

In [6]:
def preprocess_point_cloud(pcd, voxel_size):
    print(":: Downsample with a voxel size %.3f." % voxel_size)
    pcd_down = pcd.voxel_down_sample(voxel_size)

    radius_normal = voxel_size * 2
    print(":: Estimate normal with search radius %.3f." % radius_normal) 
    #Noramls are used to calculate the FPFH features
    pcd_down.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))
    return pcd_down

In [7]:
voxel_size = 0.05
source_down = preprocess_point_cloud(source, voxel_size)
target_down = preprocess_point_cloud(target, voxel_size)
print("Source has %d points while source_down has %d points" % (len(source.points), len(source_down.points)))
print("Target has %d points while target has %d points" % (len(target.points), len(target_down.points)))

:: Downsample with a voxel size 0.050.
:: Estimate normal with search radius 0.100.
:: Downsample with a voxel size 0.050.
:: Estimate normal with search radius 0.100.
Source has 307200 points while source_down has 307200 points
Target has 307200 points while target has 307200 points


In [8]:
# draw_registration_result(source_down, target_down, trans_init)

In [9]:
threshold = 0.05
criteria = o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=100000)
reg_p2pl = o3d.pipelines.registration.registration_colored_icp(
    source_down, target_down, threshold, trans_init,
    o3d.pipelines.registration.TransformationEstimationForColoredICP(),
    criteria)

In [10]:
print(reg_p2pl)
print(reg_p2pl.transformation)

RegistrationResult with fitness=1.640625e-02, inlier_rmse=3.754085e-02, and correspondence_set size of 5040
Access transformation to get result.
[[ 9.99998685e-01 -1.62193542e-03 -2.47179785e-10 -3.07658809e-02]
 [ 1.62193542e-03  9.99998685e-01 -5.49142784e-10  6.85478647e-01]
 [ 2.48070134e-10  5.48741152e-10  1.00000000e+00  1.02051258e-08]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [11]:
draw_registration_result(source, target, reg_p2pl.transformation)